<a href="https://colab.research.google.com/github/Prach1/repo/blob/master/entity_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
from os import listdir
from bs4 import BeautifulSoup
from urllib.request import urlopen

man_file = 'http://man7.org/linux/man-pages/man7/unix.7.html'
path = 'test/'
section = None
title = None

ImportError: No module named request

In [0]:
man_file = 'http://man7.org/linux/man-pages/man7/tcp.7.html'

In [0]:
def removeNonAscii(text):
	out = ''
	for c in text:
		if ord(c) in range(128):
			out += c
	return out

In [0]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
  
stop = set(stopwords.words('english')) 

def remove_punctuation(text):
  table = str.maketrans(' ', ' ', string.punctuation)
  text = text.translate(table)
  return text

def remove_stop(text):
  list_ = []
  text = text.strip().split()
  for each in text:
    if each.lower() not in stop:
      list_.append(each)
  return ' '.join(list_)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# f = open(man_file, 'r')
# man_file_contents = f.read()
# # man_file_contents = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)(\s|[A-Z].*)', man_file_contents)
# f.close()

man_file_contents = urlopen(man_file).read()

In [0]:

print('Reading file ' + man_file + '\n')

soup = BeautifulSoup(man_file_contents, 'html.parser')
h2tags = soup.find_all("h2")
title = soup.find('title').text

for i in soup.findAll('i'):
  if 'Section' in i.text:
    section = i.text
    break

all_entities_ = set()
abbreviations = []

for each in h2tags:
  des = each.find('a', id = 'DESCRIPTION')
  if des is not None:
    description_html = str(each.next_sibling)
    description_html = description_html[5:-6]
    description_html_sentences = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)(\s|[A-Z].*)',description_html)
#     print(description_html_sentences)

    for each in description_html_sentences:
      each = removeNonAscii(each)
      each = re.sub(' +',' ',each)
      each = each.strip().replace('\n','').replace('\t', '').replace('&lt;','<').replace('&gt;','>')

      if each != '':
        all_i = []
        all_b = []
        all_href = []

        soup = BeautifulSoup(each, 'html.parser')
        for i in soup.find_all('i'):
          i = i.text.split()
          all_i.append(' '.join(i))

        for b in soup.find_all('b'):
          b = b.text.split()
          all_b.append(' '.join(b))

        for href in soup.find_all('href'):
          all_href.append(href.text)

        all_caps_single = filter(None, [x for x in re.findall(r"[^\b]([A-Z]+[\s_][A-Z]*)\b", each)])
        all_mentions = filter(None, [x for x in re.findall(r"\w+\([0-9]+\)", each)])
        all_sys_files = filter(None, [x for x in re.findall(r"(/?[A-Za-z0-9.]+/[A-Za-z0-9.]+)+", each)])
        all_first_caps = filter(None, [x for x in re.findall(r"([A-Z]+[a-z0-9_]+)+", each)])
        print(each)

        all_entities_ |= set(all_i) | set(all_b) | set(all_href) | set(all_caps_single) | set(all_first_caps)
        
        print(list(all_first_caps))
        
        all_entities = []
        for each in all_entities_:
          each = remove_punctuation(each)
          each = remove_stop(each)
          all_entities.append(each)

        sent = each
        match = re.findall(r"[(][A-Z]+[)]", sent)
        flag = 0
        i = 0
        for m in match:
          index = sent.index(m)
          word = ''
          m = m.replace('(', '').replace(')', '')
          for char in m:
            try:
              i = sent.index(char, i, index)
              j = i
              while (sent[j] not in ('(', ' ', '<')):
                word += sent[j]
                j += 1
              word += ' '
            except:
              flag = 1
              break
          if not flag:
            abbreviations.append((word, match))
          else:
            break
 

Reading file http://man7.org/linux/man-pages/man7/tcp.7.html

This is an implementation of the TCP protocol defined in RFC 793, RFC 1122 and RFC 2001 with the NewReno and SACK extensions.
[]
It provides a reliable, stream-oriented, full-duplex connection between two sockets on top of <a href="../man7/ip.7.html">ip(7)</a>, for both v4 and v6 versions.
[]
TCP guarantees that the data arrives in order and retransmits lost packets.
[]
It generates and checks a per-packet checksum to catch transmission errors.
[]
TCP does not preserve record boundaries.
[]
A newly created TCP socket has no remote or local address and is not fully specified.
[]
To create an outgoing TCP connection use <a href="../man2/connect.2.html">connect(2)</a> to establish a connection to another TCP socket.
[]
To receive new incoming connections, first <a href="../man2/bind.2.html">bind(2)</a> the socket to a local address and port and then call <a href="../man2/listen.2.html">listen(2)</a> to put the socket into the l

In [0]:
if title:
	print('TITLE ' + title)
if section:
	print('SECTION ' + section)

print('ENTITIES')  
for entity in all_entities:
  if len(entity.strip()) > 2:
    print(entity)

print('ABBREVIATIONS:')
print(abbreviations)		

TITLE udp(7) - Linux manual page
ENTITIES
Locally
SOBSDCOMPAT
See
Error
udpwmemmin
UDP
INADDRANY
MSGERRQUEUE
proc interfaces
TIOCOUTQ
IPPROTOUDP
System
Socket options
FIONREAD
Using
Protocol
MSGDONTROUTE
Datagram
Datagrams
Ioctls
IPv4
Path
Unit
max
Warning
UDP
min
tcpmem
error
Returns
MSG
MTU
Defaults
ioctltype
Address format
int
IPRECVERR
MSGTRUNC
Number
udpsocket
MSGWAITALL
ioctl
Support
Transmission
udprmemmin
optval
Address
Following
Maximum
MTUDISCOVER
Unless
SIOCINQ
Linux
PAGESIZE
However
IPMTUDISCOVER
Supported
Socket
RFC
Error handling
procsysnetipv4
SOLSOCKET
sockaddrin
User
udpmem
SOL
proc
procsysnetipv4iplocalportrange
IPPROTO
Gets
EPROTO
SOBSDCOMPAT SOLSOCKET
INADDR
value
Packets
UDPCORK
procsysnetipv4ipnopmtudisc
SIOCOUTQ
Otherwise
RFC 1122
MTU
EMSGSIZE
Minimal
Warning
BSD
PAGE
pressure
ABBREVIATIONS:
[]
